Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

vect = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
clf = XGBClassifier(max_depth=5)

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [13]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [112]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5, 10),
     'clf__early_stopping_rounds':[25],
     'clf__eval_set' : [eval_set]
    }

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.4min finished


[19:55:27] WARNING: /workspace/src/learner.cc:480: 
Parameters: { early_stopping_rounds, eval_set } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [16]:
from sklearn.model_selection import train_test_split

In [94]:
X_train, X_test = train_test_split(train, stratify=train['ratingCategory'])

In [95]:
X_vect = vect.fit_transform(X_train['description'], X_train['ratingCategory'])

In [96]:
eval_set = (vect.transform(X_test['description']), X_test['ratingCategory'])

In [109]:
X_clf = clf.fit(X_vect, X_train['ratingCategory'],  early_stopping_rounds=25, eval_set = [eval_set])

[0]	validation_0-merror:0.30528
Will train until validation_0-merror hasn't improved in 25 rounds.
[1]	validation_0-merror:0.29452
[2]	validation_0-merror:0.29941
[3]	validation_0-merror:0.28963
[4]	validation_0-merror:0.28963
[5]	validation_0-merror:0.29256
[6]	validation_0-merror:0.28767
[7]	validation_0-merror:0.29550
[8]	validation_0-merror:0.29158
[9]	validation_0-merror:0.28376
[10]	validation_0-merror:0.28474
[11]	validation_0-merror:0.28082
[12]	validation_0-merror:0.28278
[13]	validation_0-merror:0.28180
[14]	validation_0-merror:0.28474
[15]	validation_0-merror:0.28278
[16]	validation_0-merror:0.28180
[17]	validation_0-merror:0.28376
[18]	validation_0-merror:0.28278
[19]	validation_0-merror:0.28180
[20]	validation_0-merror:0.28278
[21]	validation_0-merror:0.28180
[22]	validation_0-merror:0.27886
[23]	validation_0-merror:0.27691
[24]	validation_0-merror:0.27789
[25]	validation_0-merror:0.27886
[26]	validation_0-merror:0.27984
[27]	validation_0-merror:0.28082
[28]	validation_0-m

In [107]:
X_clf.score(*eval_set)

0.7299412915851272

In [116]:
grid_search.best_score_

0.724253859758853

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [31]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [32]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [33]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [143]:
subNumber = 0

In [35]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [126]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='arpack',
                   n_iter=10)

In [128]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [130]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [131]:
params = { 
    'lsi__svd__n_components': [100],
    'lsi__vect__max_df':[.9, 1.0],
    'clf__n_estimators':[5,10]
}

In [133]:
grid_search = GridSearchCV(pipe,params, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  1.8min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [134]:
grid_search.best_score_

0.7193544946525705

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [135]:
from scipy.stats import uniform, truncnorm, randint

In [136]:


rg = Generator
parameters = {
    'lsi__svd__n_components': randint(100, 250),
 #   'vect__max_df': (0.75, 1.0),
    'clf__max_depth': randint(5, 16)
}

search = RandomizedSearchCV(pipe,parameters, cv=3, verbose=1, n_iter=5, n_jobs=-1)

In [137]:
search.fit(train['description'], train['ratingCategory'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   29.5s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [82]:
search.best_score_

0.7352569068045102

### Make a Submission File

In [138]:
# Predictions on test sample
pred = search.predict(test['description'])

In [140]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [141]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [142]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

NameError: name 'subNumber' is not defined

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [3]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [4]:
# Continue Word Embedding Work Here
import spacy

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [7]:
from multiprocessing import Pool, cpu_count

In [8]:
cpu_count()

4

In [9]:
from joblib import Parallel, delayed

In [58]:
def get_vector(doc):
    return nlp(doc).vector

In [68]:
from functools import partial
def get_word_vectors_multi(docs):
    executor = Parallel(4, backend="multiprocessing" )
    do = delayed(get_vector)
    tasks = (do(doc)for doc in docs)
    return  executor(tasks)

In [76]:
vectors = get_word_vectors_multi(train['description'])

In [77]:
vectors2 = [nlp(doc).vector for doc in train['description']]

In [86]:
vec_df = pd.DataFrame(vectors)

In [87]:
vec_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.046755,0.217486,-0.139249,-0.086381,0.108063,0.082181,-0.069926,-0.078897,-0.046305,1.675343,...,-0.163038,0.011581,-0.037443,0.014240,-0.143216,0.018097,0.000748,-0.209037,-0.072346,0.042847
1,-0.019244,0.206962,-0.056247,-0.029087,0.079248,-0.001398,-0.024880,-0.057035,-0.025045,1.641834,...,-0.161042,0.091476,-0.045018,-0.110397,-0.127780,0.029475,0.054958,-0.138565,-0.067064,0.065419
2,0.046398,0.189816,-0.111451,-0.031625,0.081704,-0.025362,-0.026995,-0.062688,-0.021074,1.624273,...,-0.109123,0.022506,-0.044898,-0.077540,-0.114823,0.008062,0.054627,-0.146851,-0.135264,0.072096
3,-0.064541,0.224933,-0.091545,-0.078598,0.032475,0.032356,-0.103998,-0.050203,-0.022012,1.528916,...,-0.110060,0.047583,-0.052992,-0.065310,-0.171562,0.033367,0.090011,-0.240344,-0.095537,0.113190
4,-0.092036,0.261761,-0.188370,-0.070761,0.026548,0.098266,-0.133690,-0.031325,-0.051093,1.566256,...,-0.128262,0.054398,-0.086081,-0.126515,-0.222649,0.035877,0.110670,-0.330467,-0.044748,0.109515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4082,-0.036006,0.181084,-0.116293,0.000175,0.073497,-0.013466,-0.076238,-0.058621,-0.007652,1.623376,...,-0.129761,0.055582,-0.040615,-0.124164,-0.112927,0.017798,0.052550,-0.166922,-0.090172,0.096925
4083,-0.088918,0.201655,-0.096916,-0.118797,0.052546,0.012420,-0.104187,-0.044063,0.011607,1.583026,...,-0.086177,0.059790,-0.104474,-0.120114,-0.194738,-0.000487,0.045735,-0.261692,-0.074166,0.106413
4084,-0.079661,0.199894,-0.151530,-0.075726,0.122054,0.042213,-0.049492,-0.041022,0.002298,1.565753,...,-0.170176,0.098546,-0.064325,-0.045526,-0.192249,0.004488,0.163523,-0.255338,-0.070188,0.076157
4085,-0.024457,0.150048,-0.054028,-0.052898,0.023907,0.104424,-0.007025,-0.071171,-0.048757,1.399617,...,-0.078496,0.061411,-0.078165,-0.111210,-0.207339,0.072083,0.093042,-0.233129,-0.057862,0.055731


### Make a Submission File

In [88]:
clf.fit(vec_df, train['ratingCategory'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [90]:
test_vec = vectors = get_word_vectors_multi(test['description'])

In [91]:
# Predictions on test sample
pred = clf.predict(pd.DataFrame(test_vec))

In [92]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [93]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [97]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber = 3

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?